## FLOOD DETECTION SCENARIO PIPELINE

In [ ]:
import digitalhub as dh
PROJECT_NAME = "landslide-monitoring-v1"
proj = dh.get_or_create_project(PROJECT_NAME)

### Download data from Sentinel 1 (Orbit Direction Ascending/Descending)

Register to the open data space copernicus(if not already) and get your credentials.

https://identity.dataspace.copernicus.eu/auth/realms/CDSE/login-actions/registration?client_id=cdse-public&tab_id=FIiRPJeoiX4

Log the credentials as project secret keys as shown below

In [ ]:
# THIS NEED TO BE EXECUTED JUST ONCE
secret0 = proj.new_secret(name="CDSETOOL_ESA_USER", secret_value="esa_username")
secret1 = proj.new_secret(name="CDSETOOL_ESA_PASSWORD", secret_value="esa_password")

In [ ]:
function_s1 = proj.new_function("download_images_s1",kind="container",image="ghcr.io/tn-aixpa/sentinel-tools:0.11.6",command="python")

### Download Sentinel 1

In [ ]:
function_s1 = proj.new_function("download_images_s1",kind="container",image="ghcr.io/tn-aixpa/sentinel-tools:0.11.6",command="python")

### Log artifact

Log the shape file 'Shapes_TN' which can be downloaded from the [WebGIS Portal](https://webgis.provincia.tn.it/) from https://siatservices.provincia.tn.it/idt/vector/p_TN_377793f1-1094-4e81-810e-403897418b23.zip. Unzip the files in a folder named 'Shapes_TN' and then log it


In [ ]:
artifact_name='Shapes_TN'
src_path='/Shapes_TN'
artifact_data = proj.log_artifact(name=artifact_name, kind="artifact", source=src_path)

Note that to invoke the operation on the platform, the data should be avaialble as an artifact on the platform datalake.


In [ ]:
artifact = proj.get_artifact("Shapes_TN")
artifact.key

The resulting datasets will be registered as the project artifact in the datalake under the name 'Shapes_TN'

### Elaboration

In [ ]:
function_rs = proj.new_function("elaborate",kind="container", image="ghcr.io/tn-aixpa/rs-landslide-monitoring:2.9", command="/bin/bash", code_src="launch.sh")

### Pipeline

In [ ]:
%%writefile "landslide_pipeline.py"

from digitalhub_runtime_kfp.dsl import pipeline_context

def myhandler(startDate, endDate, geometry, outputName):
    with pipeline_context() as pc:
        
        s1_ascending = "s1_ascending_" + str(outputName)
        s1_descending = "s1_descending_"+ str(outputName) 
    
        string_dict_data_asc = """{"satelliteParams":{"satelliteType": "Sentinel1","processingLevel": "LEVEL1","sensorMode": "IW","productType": "SLC","orbitDirection": "ASCENDING","relativeOrbitNumber": "117"},"startDate": \"""" + str(startDate) + """\","endDate": \"""" + str(endDate) + """\","geometry": \"""" + str(geometry) + """\","area_sampling": "True","artifact_name": \"""" + str(s1_ascending) + """\"}"""
        string_dict_data_des = """{"satelliteParams":{"satelliteType": "Sentinel1","processingLevel": "LEVEL1","sensorMode": "IW","productType": "SLC","orbitDirection": "DESCENDING","relativeOrbitNumber": "168"},"startDate": \"""" + str(startDate) + """\","endDate": \"""" + str(endDate) + """\","geometry": \"""" + str(geometry) + """\","area_sampling": "True","artifact_name": \"""" + str(s1_descending) + """\"}"""
        
        s1 = pc.step(name="download-asc",
                     function="download_images_s1",
                     action="job",
                     secrets=["CDSETOOL_ESA_USER","CDSETOOL_ESA_PASSWORD"],
                     fs_group='8877',
                     args=["main.py", string_dict_data_asc],
                     volumes=[{
                        "volume_type": "persistent_volume_claim",
                        "name": "volume-land",
                        "mount_path": "/app/files",
                        "spec": { "size": "300Gi" }
                        }
                    ])

        s2 = pc.step(name="download-desc",
                     function="download_images_s1",
                     action="job",
                     secrets=["CDSETOOL_ESA_USER","CDSETOOL_ESA_PASSWORD"],
                     fs_group='8877',
                     args=["main.py", string_dict_data_des],
                     volumes=[{
                        "volume_type": "persistent_volume_claim",
                        "name": "volume-land",
                        "mount_path": "/app/files",
                        "spec": { "size": "300Gi" }
                        }
                    ]).after(s1)
        
        s3 = pc.step(name="elaborate",
                     function="elaborate",
                     action="job",
                     fs_group='8877',
                     resources={"cpu": {"requests": "6", "limits": "12"},"mem":{"requests": "32Gi", "limits": "64Gi"}},
                     volumes=[{
                        "volume_type": "persistent_volume_claim",
                        "name": "volume-land",
                        "mount_path": "/app/data",
                        "spec": { "size": "500Gi" }
                    }],
                     args=['/shared/launch.sh', str(s1_ascending), str(s1_descending), str(startDate), str(endDate), str(outputName), 'Shapes_TN', 'ammprv_v.shp', 'Map',  str(geometry)]
                     ).after(s2)
     


Create workflow using project repo source file

In [ ]:
workflow = proj.new_workflow(name="pipeline_landslide", kind="kfp", code_src= "landslide_pipeline.py", handler = "myhandler")

Build workflow

In [ ]:
wfbuild = workflow.run(action="build", wait=True)

Run workflow

In [ ]:
workflow_run = workflow.run(action="pipeline", parameters={
    "startDate": "2020-11-01",
    "endDate": "2021-11-07",
    "geometry": "POLYGON ((10.595369 45.923394, 10.644894 45.923394, 10.644894 45.945838, 10.595369 45.945838, 10.595369 45.923394))",
    "outputName": "landslide_2020-11-01_2020-11-07"
    })